# Assignment

## Brief

Write the Python codes for the following questions.

## Instructions

Paste the answer as Python in the answer code section below each question.

### Question 1

Question: Implement a simple Thrift server and client that defines a `Student` struct with fields `name` (string), `age` (integer), and `courses` (list of strings). Include a service `School` with a method `enrollCourse` that takes a `Student` record and a course name, adds the course to the student's course list, and returns the updated `Student` record.

Answer:

#### Thrift schema (student.thrift)

In [4]:
%%writefile ../schema/student.thrift

struct Student {
  1: required string userName,
  2: optional i64 age,
  3: optional list<string> courses
}

service School {
    Student enrollCourse(1: required Student student, 2: required string course)
}

Overwriting ../schema/student.thrift


#### Thrift server (student_server.py)

In [5]:
%%writefile ../student_server.py
import thriftpy2
student_thrift = thriftpy2.load("./schema/student.thrift", module_name="student_thrift")

from thriftpy2.rpc import make_server

class School(object):
    def enrollCourse(self, student, course):
        student.courses.append(course)
        return student

server = make_server(student_thrift.School, School(), client_timeout=None)
server.serve()

Overwriting ../student_server.py


Then, run `python student_server.py` in a new terminal. This will start the server.

#### Thrift client (student_client.py)

In [6]:
import thriftpy2
student_thrift = thriftpy2.load("../schema/student.thrift", module_name="student_thrift")

from thriftpy2.rpc import make_client

school = make_client(student_thrift.School, timeout=None)

In [8]:
martin = student_thrift.Student(
    userName="Martin", age=14, courses=["history", "math"]
)

In [9]:
martin.courses

['history', 'math']

### Question 2

Question: Implement a simple Protocol Buffers server and client that defines a `Book` message with fields `title` (string), `author` (string), and `page_count` (integer). Include a service `Library` with a method `checkoutBook` that takes a `Book` message and returns the same `Book` message.

Answer:

#### Protobuf schema (book.proto)

In [1]:
%%writefile ../schema/book.proto
syntax = "proto3";

message Book {
  string title = 1;
  string author = 2;
  int32 page_count = 3;
}


service Library {
  rpc checkoutBook(Book) returns (Book) {}
}

Writing ../schema/book.proto


python -m grpc_tools.protoc -I./schema --python_out=. --grpc_python_out=. ./schema/book.proto

#### Protobuf server (book_server.py)

In [2]:
%%writefile ../book_protobuf_server.py
import grpc
from concurrent import futures

import book_pb2 as pb
import book_pb2_grpc as pb_grpc


class LibraryService(pb_grpc.LibraryServicer):
    def checkoutBook(self, request, context):
        # A simple echo: return the same Book
        book = pb.Book()
        book.CopyFrom(request)
        return book

def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=4))
    pb_grpc.add_LibraryServicer_to_server(LibraryService(), server)
    server.add_insecure_port("[::]:50051")
    server.start()
    print("Library gRPC server listening on :50051")
    server.wait_for_termination()

if __name__ == "__main__":
    serve()

Writing ../book_protobuf_server.py


python book_protobuf_server.py

#### Protobuf client (book_client.py)

In [5]:
import sys
sys.path.append('..')
import grpc
import book_pb2 as pb
import book_pb2_grpc as pb_grpc

def run():
    with grpc.insecure_channel("localhost:50051") as channel:
        stub = pb_grpc.LibraryStub(channel)
        book = pb.Book(title="The Folded World", author="B3", page_count=312)
        checked_out = stub.checkoutBook(book)
        print("Checked out:")
        print(f"  Title      : {checked_out.title}")
        print(f"  Author     : {checked_out.author}")
        print(f"  Page count : {checked_out.page_count}")

if __name__ == "__main__":
    run()

Checked out:
  Title      : The Folded World
  Author     : B3
  Page count : 312


## Submission

- Submit the URL of the GitHub Repository that contains your work to NTU black board.
- Should you reference the work of your classmate(s) or online resources, give them credit by adding either the name of your classmate or URL.